In [1]:
import os
import tqdm

import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score, pairwise_distances, silhouette_samples
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.neighbors import NearestNeighbors

import scipy.stats as st

import umap

import igraph as ig
import leidenalg as la

import networkx as nx

import subprocess

from pynndescent import NNDescent

from csv import DictWriter

from datetime import datetime

In [2]:
def shuffle_row(row):
    shuffled_row = row.values.copy()
    np.random.shuffle(shuffled_row)
    return pd.Series(shuffled_row, index=row.index)

def shuffle_rows(df):
    columns_to_shuffle = df.columns[1:]
    df[columns_to_shuffle] = df[columns_to_shuffle].apply(shuffle_row, axis=1)
    return df

In [3]:
def get_geom_mean_expression(expression_df):
    """
    
    Function to take an expression dataframe from the microarrays and collapse it into the means of
    all replicate chips.
    """
    # C2 and S12 got removed during quality control
    x = [
        'Ll', 
        'Lm', 
        'Lh', 
        'S0', 
        'S3', 
        'S6', 
        'S9', 
        # 'S12', 
        'S15', 
        'S24', 
        'C0', 
        # 'C2', 
        'C4', 
        'C6', 
        'C8', 
        'C10', 
        'C12', 
        'C14', 
        'C16', 
        'C18']
    
    # cols = expression_df.columns[1:]
    # x = [c for c in x if c in cols]
    
    condition_expr_dict = {c.split("_")[0]: [] for c in expression_df.columns[1:]}
    
    for c in list(expression_df.columns)[1:]:
        
        cond = c.split('_')[0]
        if cond in condition_expr_dict.keys():
            expr_list = condition_expr_dict.get(cond, [])

            # Need to avoid true zeros
            expr_list.append(expression_df[c].values)
            condition_expr_dict[cond] = expr_list
        
    condition_mean_dict = {c: (st.mstats.gmean(np.array(condition_expr_dict[c]) + 1, 0) - 1) for c in condition_expr_dict.keys() if c in x}
    
    mean_expr_df = pd.DataFrame(condition_mean_dict)
    mean_expr_df['TTHERM_ID'] = expression_df['TTHERM_ID'].values
    cols = list(mean_expr_df.columns)
    reorder = cols[-1:] + cols[:-1]
    mean_expr_df = mean_expr_df[reorder]
    
    return mean_expr_df

def normalizer(array):
    """
    Normalizes the values of an array to range from zero to one
    """
    
    a = np.array(array)
    
    normalized = (array - np.min(array)) / (np.max(array) - np.min(array))
    
    return normalized

def normalize_expression_per_gene(expression_df):
    """
    Function to normalize all gene expression to range from zero to one.
    """
    if 'TTHERM_ID' in expression_df.columns:
        ttids = expression_df['TTHERM_ID'].values
        data = expression_df[list(expression_df.columns)[1:]]
        
        norm_expression_df = data.apply(lambda row: normalizer(row), axis=1)
        norm_expression_df['TTHERM_ID'] = ttids
        
        columns = norm_expression_df.columns.tolist()
        
        rearrangment = columns[-1:] + columns[:-1]
        
        norm_expression_df = norm_expression_df[rearrangment]
        
    else:
        norm_expression_df = expression_df.apply(lambda row: normalizer(row), axis=1)
    
    return norm_expression_df

In [4]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')
go_df = pd.read_csv('../enrichment/go_annotations.csv')
kegg_df = pd.read_csv('../enrichment/kegg_annotations.csv')
ec_df = pd.read_csv('../enrichment/ec_annotations.csv')

go_df['GOs_description'].loc[go_df['GOs'] == 'GO:0000001'].values[0]

# As of 2020 https://www.ncbi.nlm.nih.gov/research/cog/
COG_dict = {
    "A" : "RNA processing and modification",
    "B" : "Chromatin structure and dynamics",
    "C" : "Energy production and conversion",
    "D" : "Cell cycle control, cell division, chromosome partitioning",
    "E" : "Amino acid transport and metabolism",
    "F" : "Nucleotide transport and metabolism",
    "G" : "Carbohydrate transport and metabolism",
    "H" : "Coenzyme transport and metabolism",
    "I" : "Lipid transport and metabolism",
    "J" : "Translation, ribosomal structure and biogenesis",
    "K" : "Transcription",
    "L" : "Replication, recombination, and repair",
    "M" : "Cell wall/membrane/envelope biogenesis",
    "N" : "Cell motility",
    "O" : "Posttranslational modification, protein turnover, chaperones",
    "P" : "Inorganic ion transport and metabolism",
    "Q" : "Secondary metabolites biosynthesis, transport and catabolism",
    "T" : "Signal transduction mechanisms",
    "U" : "Intracellular trafficking, secretion, and vesicular transport",
    "V" : "Defense mechanisms",
    "W" : "Extracellular structures",
    "X" : "Mobilome: prophages, transposons",
    "Y" : "Nuclear structure",
    "Z" : "Cytoskeleton",
    "R" : "General function prediction only",
    "S" : "Function unknown",
}
def term_count_dict_from_annotation_df(annot_df, term_column):
    
    column = annot_df[term_column].values
    
    funct_terms = []
    for entry in column:
        if entry != '-':
            if term_column == 'COG_category':
                # terms = [f'{e}: {COG_dict[e]}' for e in entry]
                terms = list(entry)
            else:
                terms = entry.split(',')
            for t in terms:
                funct_terms.append(t)

#     len(terms)
    
    term_count_dict = {}
    
    for t in funct_terms:
        count = term_count_dict.get(t, 0)
        count += 1
        term_count_dict[t] = count
        
    return term_count_dict
def enrichment_analysis(module, leiden_label_df, phases, background_annotation, term_column):
    
    module_ttids = leiden_label_df.loc[leiden_label_df[f'leiden_label_{phases}'] == module]['TTHERM_ID'].values
    
    module_annotation = background_annotation.loc[background_annotation['TTHERM_ID'].isin(module_ttids)]
    
    background_term_dict = term_count_dict_from_annotation_df(background_annotation, term_column)
    module_term_dict = term_count_dict_from_annotation_df(module_annotation, term_column)
    
    bs = []
    ps = []
    folds = []
    terms = []
    
    for t, module_count in module_term_dict.items():
        
        background_count = background_term_dict[t]
        module_size = len(module_annotation)
        background_size = len(background_annotation)
        
        standard_contingency_table = [
                                [module_count, background_count - module_count], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        # The -1 and +1 make this more conservative (see explanation from the DAVID database: 
        # https://david.ncifcrf.gov/helps/functional_annotation.html#geneenrich)
        conservative_contingency_table = [
                                [module_count - 1, background_count - module_count + 1], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        
        odds, p_standard = st.fisher_exact(standard_contingency_table, 'greater')
        odds, p_conservative = st.fisher_exact(conservative_contingency_table, 'greater')
        
        p_reasonable = np.mean([p_standard, p_conservative])
        
        bonferroni  = p_reasonable * len(module_term_dict)

        fold_enrichment = (module_count/module_size) / (background_count/background_size)

        if bonferroni <= 0.05:
            
            ps.append(p_reasonable)
            bs.append(bonferroni)
            folds.append(fold_enrichment)
            terms.append(t)
            
#         else:
#             ps.append('')
#             bs.append('')
#             folds.append('')
#             terms.append('')
            
    return ps, bs, folds, terms
            
            
def get_GO_info(go_term):
    
    name = go_df['GOs_description'].loc[go_df['GOs'] == go_term].values[0]
    
    definition = go_df['GOs_definition'].loc[go_df['GOs'] == go_term].values[0]
    
    obsolete = go_df['GOs_obsolete'].loc[go_df['GOs'] == go_term].values[0]
    
    return name, definition, obsolete

def get_KEGG_info(term):
    return kegg_df['KEGG_ko_description'].loc[kegg_df['KEGG_ko'] == term].values[0]

def get_EC_info(term):
    return ec_df['EC_description'].loc[ec_df['EC'] == term].values[0]

def get_enrichment_df(lldf, phases, background_annotation, term_columns=['COG_category', 'GOs', 'KEGG_ko', 'EC'], outfile=None):
    
    module_dfs = []
    
    for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):
        
        term_dfs = []

        for tc in term_columns:
        
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, background_annotation, tc)

            # fl.write(f'Module {m}\n')
            
            info = []

            if tc == 'GOs':

                for t in terms:
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        info.append(f'{name.capitalize()}: {definition} (obsolete)')
                    else:
                        info.append(f'{name.capitalize()}: {definition}')
                        
            elif tc == 'COG_category':
                for t in terms:
                    info.append(COG_dict[t])
                                
            elif tc == 'KEGG_ko':
                for t in terms:
                    info.append(get_KEGG_info(t))
                    
            elif tc == 'EC':
                for t in terms:
                    info.append(get_EC_info(t))
                    
            term_df = pd.DataFrame({'module': [m]*len(terms),
                                    'term': terms,
                                    'info': info,
                                    'fold_change': folds,
                                    'bonferroni': bs})
            
            term_dfs.append(term_df)
            
        module_df = pd.concat(term_dfs)
        
        module_dfs.append(module_df)
        
    all_enrichment_df = pd.concat(module_dfs)
    
    if outfile:
        all_enrichment_df.to_csv(outfile, index=False)
    
    return all_enrichment_df

In [5]:
partition_type = 'EXP'
full_filtered_df = pd.read_csv('../microarray_probe_alignment_and_filtering/allgood_filt_agg_tidy_2021aligned_qc_rma_expression_full.csv')
full_filtered_df = full_filtered_df.rename(columns={'Unnamed: 0': 'TTHERM_ID'})

In [6]:
full_filtered_df.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_000000042,6.928782,7.264201,6.934214,6.732989,6.970612,7.150978,6.126826,6.868968,6.641119,...,6.450318,8.049750,7.788162,7.052154,6.517742,6.918501,6.048861,7.041619,6.757932,5.817246
1,TTHERM_000000045,9.633489,9.977124,10.027529,9.720665,9.605762,10.225542,10.279608,10.459966,10.693337,...,11.130466,11.207738,11.009172,10.615417,11.038938,11.009222,10.216348,11.099187,11.172276,10.561021
2,TTHERM_00000010,5.066343,4.767264,5.010981,6.139047,4.619361,4.751761,5.818550,5.342529,5.483750,...,6.314438,7.423571,7.507645,7.417087,7.147801,7.747930,7.093641,7.672685,7.511290,6.890117
3,TTHERM_00000020,4.696881,4.638401,4.956299,6.942556,5.101252,4.730307,8.457690,4.526411,4.908300,...,5.250233,4.974993,5.747498,5.252167,5.210531,7.083187,5.252222,5.037613,5.495281,5.013987
4,TTHERM_00000030,4.654278,4.537105,4.928739,5.063991,4.584168,4.911880,5.935311,4.519470,4.757861,...,4.651688,4.920573,4.636333,4.883712,4.779395,4.744335,4.513140,4.838428,4.961475,4.653340


In [7]:
# full_filtered_df = shuffle_rows(full_filtered_df)
# partition_type = 'NC'

In [8]:
full_filtered_df.head()

,TTHERM_ID,Ll_GSM283687,Ll_GSM284355,Ll_GSM284362,Lm_GSM283690,Lm_GSM284357,Lm_GSM284363,Lh_GSM283691,Lh_GSM284360,Lh_GSM284364,...,C12_GSM656237,C14_GSM285580,C14_GSM285593,C14_GSM656238,C16_GSM285582,C16_GSM285595,C16_GSM656239,C18_GSM285583,C18_GSM285596,C18_GSM656240
0,TTHERM_000000042,6.928782,7.264201,6.934214,6.732989,6.970612,7.150978,6.126826,6.868968,6.641119,...,6.450318,8.049750,7.788162,7.052154,6.517742,6.918501,6.048861,7.041619,6.757932,5.817246
1,TTHERM_000000045,9.633489,9.977124,10.027529,9.720665,9.605762,10.225542,10.279608,10.459966,10.693337,...,11.130466,11.207738,11.009172,10.615417,11.038938,11.009222,10.216348,11.099187,11.172276,10.561021
2,TTHERM_00000010,5.066343,4.767264,5.010981,6.139047,4.619361,4.751761,5.818550,5.342529,5.483750,...,6.314438,7.423571,7.507645,7.417087,7.147801,7.747930,7.093641,7.672685,7.511290,6.890117
3,TTHERM_00000020,4.696881,4.638401,4.956299,6.942556,5.101252,4.730307,8.457690,4.526411,4.908300,...,5.250233,4.974993,5.747498,5.252167,5.210531,7.083187,5.252222,5.037613,5.495281,5.013987
4,TTHERM_00000030,4.654278,4.537105,4.928739,5.063991,4.584168,4.911880,5.935311,4.519470,4.757861,...,4.651688,4.920573,4.636333,4.883712,4.779395,4.744335,4.513140,4.838428,4.961475,4.653340


In [9]:
full_filtered_norm_df = normalize_expression_per_gene(full_filtered_df)
raw_data = full_filtered_norm_df[list(full_filtered_norm_df.columns)[1:]].values

In [10]:
# np.random.seed(42)
# X, _ = make_blobs(n_samples=10000, n_features=30, centers=350, cluster_std=1.0, random_state=42)  # Use only 2 features
# # Convert X to a DataFrame
# columns = ['feature' + str(i) for i in range(X.shape[1])]
# df = pd.DataFrame(X, columns=columns)
# raw_data = df.values

In [11]:
def compute_pairwise_distance_matrix(data_df, metric, n_jobs=-1):
    
    return pairwise_distances(data_df, metric=metric, n_jobs=n_jobs)

In [12]:
def compute_nns(data_df, nn, metric, random_state=42, n_jobs=-1):
    
    # num_neighbors = NearestNeighbors(n_neighbors=nn, metric='precomputed', n_jobs=-1).fit(manhattan_distance_matrix)
    # nn_dists, nn_idxs = num_neighbors.kneighbors(return_distance=True)

    n_trees = min(64, 5 + int(round((raw_data.shape[0]) ** 0.5 / 20.0)))
    n_iters = max(5, int(round(np.log2(raw_data.shape[0]))))
    knn_search_index = NNDescent(
                data_df,
                n_neighbors=nn,
                metric=metric,
                # metric_kwds=metric_kwds,
                random_state=random_state,
                n_trees=n_trees,
                n_iters=n_iters,
                max_candidates=60,
                # low_memory=low_memory,
                n_jobs=n_jobs,
                verbose=False,
                compressed=False,
            )
    nn_idxs, nn_dists = knn_search_index.neighbor_graph

    return nn_idxs, nn_dists

In [13]:
def compute_umap_graph(data_df, nn, metric, nn_idxs, nn_dists):
    
    result, sigmas, rhos, dists = umap.umap_.fuzzy_simplicial_set(data_df, nn, 42, metric, knn_indices=nn_idxs, knn_dists=nn_dists, return_dists=True)

    sources, targets = result.nonzero()
    edge_list = zip(sources, targets)
    weights = result.data

    g = ig.Graph(edges=edge_list, edge_attrs={'weight': weights})
    
    return g

In [14]:
def compute_leiden_partition(graph, resolution_parameter, random_state=42):
        
        partition = la.find_partition(graph, la.CPMVertexPartition, resolution_parameter = resolution_parameter, seed=random_state, weights='weight')
        # partition = la.find_partition(g, la.ModularityVertexPartition, seed=42, weights='weight')

        leiden_modules = np.array(partition.membership)

        return leiden_modules

In [15]:
def compute_communities(parition, idx_labels):
    communities = {}

    for idx, membership in enumerate(parition):
        if membership not in communities:
            communities[membership] = []
        communities[membership].append(idx_labels[idx])

    return communities

In [16]:
def compute_silhouette_score(distance_matrix, parition):
    return silhouette_score(distance_matrix, parition, metric='precomputed')

In [17]:
def compute_modularity(graph, communities):
    nx_g = nx.Graph(graph.get_edgelist())
    return nx.community.quality.modularity(nx_g, communities, weight='weight')

In [18]:
def format_parition_for_enrichment(df, parition):
    edf = pd.DataFrame.from_dict({'TTHERM_ID': []})
    edf['TTHERM_ID'] = df['TTHERM_ID'].values
    edf[f'leiden_label_full'] = parition
    return edf

In [19]:
def remove_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)

In [20]:
def compute_enrichment(df, parition):
    edf = format_parition_for_enrichment(df, parition)

    temp_scan_file = './temp_scan_partition.csv'

    temp_enrich_file = './temp_scan_enrich.csv'

    edf.to_csv(temp_scan_file, index=False)

    subprocess.run(['python3', './fast_enrichment_analysis.py', temp_scan_file, temp_enrich_file])

    cedf = pd.read_csv(temp_enrich_file)
    
    remove_file(temp_scan_file)

    remove_file(temp_enrich_file)

    return cedf

In [21]:
def compute_num_clusters(parition, communities=None):
    if communities is None:
        return len(set(parition))
    
    if len(set(parition)) != len(communities):
        raise ValueError(f'The number of clusters/modules ({len(set(parition))}) in the parition != the number of communities ({len(communities)}).')
    
    return len(set(parition))

In [22]:
def compute_cluster_sizes(communities):
    return [len(community) for community in communities.values()]

In [23]:
def compute_enriched_cluster_sizes(communities, cedf):
    enriched_cluster_mods = set(cedf['module'].values)
    return [len(community) for mod, community in communities.items() if mod in enriched_cluster_mods]

In [24]:
def compute_cluster_size_mean(cluster_sizes):
    return np.mean(cluster_sizes)

def compute_cluster_size_median(cluster_sizes):
    return np.median(cluster_sizes)

def compute_cluster_size_sd(cluster_sizes):
    return np.std(cluster_sizes)

def compute_cluster_size_sd(cluster_sizes):
    return np.std(cluster_sizes)

In [25]:
def compute_num_enriched_clusters(cedf):
    return len(set(cedf['module'].values))

In [26]:
def compute_num_enriched_cluster_genes(edf, parition):
    total_num_genes = 0

    for m in set(edf['module'].values):
        num_genes = np.count_nonzero(parition == int(m))
        total_num_genes += num_genes
    
    return total_num_genes
    

In [27]:
def write_to_csv(csv_file_path, data_item, header):
    # Check if the CSV file exists and write header if it doesn't
    if not os.path.isfile(csv_file_path):
        with open(csv_file_path, 'w', newline='') as file:
            writer = DictWriter(file, fieldnames=header)
            writer.writeheader()

    with open(csv_file_path, 'a', newline='') as file:
        writer = DictWriter(file, fieldnames=header)
        writer.writerow(data_item)

# SCAN START

In [28]:
curr_datetime = str(datetime.now())

In [29]:
idx_labels = list(range(raw_data.shape[0]))

metric = 'manhattan'
n_jobs = -1
random_state = 42

In [30]:
scan_nns = np.arange(2, 13, 1)
# scan_nns = [3]
scan_nns

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [31]:
# scan_rps = np.arange(0.005, 1.1, 0.005)
scan_rps = np.arange(0.1, 1.1, 0.1)
# scan_rps = [0.6]
scan_rps

array([0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 , 0.045,
       0.05 , 0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085, 0.09 ,
       0.095, 0.1  , 0.105, 0.11 , 0.115, 0.12 , 0.125, 0.13 , 0.135,
       0.14 , 0.145, 0.15 , 0.155, 0.16 , 0.165, 0.17 , 0.175, 0.18 ,
       0.185, 0.19 , 0.195, 0.2  , 0.205, 0.21 , 0.215, 0.22 , 0.225,
       0.23 , 0.235, 0.24 , 0.245, 0.25 , 0.255, 0.26 , 0.265, 0.27 ,
       0.275, 0.28 , 0.285, 0.29 , 0.295, 0.3  , 0.305, 0.31 , 0.315,
       0.32 , 0.325, 0.33 , 0.335, 0.34 , 0.345, 0.35 , 0.355, 0.36 ,
       0.365, 0.37 , 0.375, 0.38 , 0.385, 0.39 , 0.395, 0.4  , 0.405,
       0.41 , 0.415, 0.42 , 0.425, 0.43 , 0.435, 0.44 , 0.445, 0.45 ,
       0.455, 0.46 , 0.465, 0.47 , 0.475, 0.48 , 0.485, 0.49 , 0.495,
       0.5  , 0.505, 0.51 , 0.515, 0.52 , 0.525, 0.53 , 0.535, 0.54 ,
       0.545, 0.55 , 0.555, 0.56 , 0.565, 0.57 , 0.575, 0.58 , 0.585,
       0.59 , 0.595, 0.6  , 0.605, 0.61 , 0.615, 0.62 , 0.625, 0.63 ,
       0.635, 0.64 ,

In [ ]:
scan_dict = {}

In [ ]:
distance_matrix = compute_pairwise_distance_matrix(raw_data, metric, n_jobs)

In [ ]:
for idx, nn in enumerate(scan_nns):     
    print(idx+1,'of',len(scan_nns))     
    print('NNs: ', nn)

    scan_dict[nn] = {}

    nn_idxs, nn_dists = compute_nns(raw_data, nn, metric, random_state, n_jobs)
    scan_dict[nn]['nn_idxs'] = nn_idxs
    scan_dict[nn]['nn_dists'] = nn_dists

    nn_graph = compute_umap_graph(raw_data, nn, metric, nn_idxs, nn_dists)
    scan_dict[nn]['nn_graph'] = nn_graph

    for rp in tqdm.tqdm(scan_rps):

        scan_dict[nn][rp] = {}
        
        parition = compute_leiden_partition(nn_graph, rp, random_state)
        scan_dict[nn][rp]['partition'] = parition

        communities = compute_communities(parition, idx_labels)
        scan_dict[nn][rp]['communities'] = communities

        sil_score = compute_silhouette_score(distance_matrix, parition)
        scan_dict[nn][rp]['sil_score'] = sil_score

        modularity = compute_modularity(nn_graph, communities.values())
        scan_dict[nn][rp]['modularity'] = modularity

        enrichment_df = compute_enrichment(full_filtered_norm_df, parition)
        scan_dict[nn][rp]['enrichment_df'] = enrichment_df

        num_clusters = compute_num_clusters(parition, communities.values())
        scan_dict[nn][rp]['num_clusters'] = num_clusters

        num_enriched_clusters = compute_num_enriched_clusters(enrichment_df)
        scan_dict[nn][rp]['num_enriched_clusters'] = num_enriched_clusters

        num_enriched_cluster_genes = compute_num_enriched_cluster_genes(enrichment_df, parition)
        scan_dict[nn][rp]['num_enriched_cluster_genes'] = num_enriched_cluster_genes

        cluster_sizes = compute_cluster_sizes(communities)
        scan_dict[nn][rp]['cluster_sizes'] = cluster_sizes

        enriched_cluster_sizes = compute_enriched_cluster_sizes(communities, enrichment_df)
        scan_dict[nn][rp]['enriched_cluster_sizes'] = enriched_cluster_sizes

        cluster_stats = {
        'partition_type': partition_type,

        'dimensionality': 'baseline',

        'metric': metric,
        'graph': 'umap_fuzzy_simplicial_set',
        'nns': nn,

        'clustering': 'leiden_cpm',
        'parameter': rp,

        'silhouette_score': sil_score,
        'modularity': modularity,

        'nclusters': num_clusters,
        'mean_cluster_size': compute_cluster_size_mean(cluster_sizes),
        'median_cluster_size': compute_cluster_size_median(cluster_sizes),
        'sd_cluster_size': compute_cluster_size_sd(cluster_sizes),

        'nenriched_clusters': num_enriched_clusters,
        'mean_enriched_cluster_size': compute_cluster_size_mean(enriched_cluster_sizes),
        'median_enriched_cluster_size': compute_cluster_size_median(enriched_cluster_sizes),
        'sd_enriched_cluster_size': compute_cluster_size_sd(enriched_cluster_sizes),
        'nenriched_cluster_genes': num_enriched_cluster_genes,

        'datetime': curr_datetime
        }

        write_to_csv('./scan_stats_v1.csv', cluster_stats, list(cluster_stats.keys()))

In [33]:
pd.read_csv('./scan_stats_v1.csv')

,partition_type,dimensionality,metric,graph,nns,clustering,parameter,silhouette_score,modularity,nclusters,mean_cluster_size,median_cluster_size,sd_cluster_size,nenriched_clusters,mean_enriched_cluster_size,median_enriched_cluster_size,sd_enriched_cluster_size,nenriched_cluster_genes,datetime
0,EXP,baseline,manhattan,umap_fuzzy_simplicial_set,3,leiden_cpm,0.600,0.024209,0.422344,7759,2.619667,2.0,0.842902,491,3.034623,3.0,0.856759,1490,2024-02-02 14:43:14.355357
1,EXP,baseline,clr,umap_fuzzy_simplicial_set,12,leiden_cpm,0.095,-0.016423,0.294406,1905,10.669816,10.0,5.476522,316,12.841772,12.0,5.938527,4058,2024-02-02 15:37:14.079288
2,EXP,baseline,clr,umap_fuzzy_simplicial_set,12,leiden_cpm,0.095,0.020106,0.299335,1839,11.052746,10.0,5.651367,301,14.926910,13.0,7.206584,4493,2024-02-02 15:47:45.611361
3,EXP,baseline,clr,umap_fuzzy_simplicial_set,3,leiden_cpm,0.100,0.015219,0.536167,2714,7.489315,7.0,2.523887,345,8.631884,8.0,2.948962,2978,2024-02-02 15:50:51.602937


In [ ]:
distance_matrix.shape